In [36]:
# Testing all of the basic workflow of the package
from PYTHON.io import *
from PYTHON.Atom import *
from PYTHON.Bond import *
from PYTHON.Molecule import *
from PYTHON.Interaction import *
from PYTHON.Crystal import *

from datetime import datetime
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib qt

In [2]:
start = datetime.now()

# IO Utility

In [3]:
# Test Cif2SUpercell() method
Cif2Supercell('LAVPIK.cif',supercell_size=[[5,0,0],[0,5,0],[0,0,5]],output_path='.')

In [4]:
# Test Mol2Reader() class
mol2_reader = Mol2Reader('supercell.mol2')
mols = mol2_reader.read()

# Get Data

In [19]:
# Add rings as atoms
acenes = []
bases = []
for mol in mols:
    mol.add_rings_as_atoms()
    mol.assign_atoms_interaction_dict()
    basis, xmag = Acene(mol).get_vectors(return_xmag=True)
    bases.append(basis)
    acenes.append(Acene(mol))

In [6]:
# Calculate All Atom Distances
all_atoms = []
for mol in acenes:
    all_atoms.append([atom.coordinates for atom in mol.atoms])
all_atoms = np.array(all_atoms)
disps = []
dists = []
mol1s = []
mol2s = []
# Loop Through each molecule and atom
# Use numpy.roll to batch i to i distance, i to i+1 distance
# Rather than looping through each atom
for i, arr1 in enumerate(all_atoms[:-1]):
    for j, arr2 in enumerate(all_atoms[i+1:],i+1):
        temp_dist = []
        for x in range(len(arr2)):
            mol1s += [i]*len(arr2)
            mol2s += [j]*len(arr2)
            disp = arr2 - arr1
            dist2 = disp[:,0] * disp[:,0] + disp[:,1] * disp[:,1] + disp[:,2] * disp[:,2]
            dist = np.sqrt(dist2)
            temp_dist.append(dist)
            arr2 = np.roll(arr2,-1,axis=0)
        dists.append(temp_dist)
dists = np.array(dists)
# Put distances in order of atom indices
in_atom_order = np.array([dist.flatten('F') for dist in dists]).reshape(-1)
d1 = dists.shape[0]
d2 = dists.shape[1]
arange = np.arange(d2)
atom1s = np.concatenate([[x]*d2 for x in range(d2)]*d1)
atom2s = np.concatenate([np.roll(arange,-x) for x in range(d2)]*d1)
# Turn Atom Distances to DataFrame
data_dict= {'mol1s':mol1s,'mol2s':mol2s,'atom1s':atom1s,'atom2s':atom2s,'dists':in_atom_order}
atom_dist_df = pd.DataFrame(data_dict)

In [ ]:
# Calculate Central Molecule Atom Distances
# Atom Distances from central molecule
central_molecule, central_idx = crystal.get_central_molecule(return_idx=True)
central_cog = central_molecule.centre_of_geometry()
central_atom_coords = np.array([atom.coordinates for atom in central_molecule.atoms]) # shape = (n_atoms,3)
all_atom_coords = []
for mol in crystal.molecules:
    all_atom_coords.append(np.array([atom.coordinates for atom in mol.atoms]))
all_atom_coords = np.array(all_atom_coords) # shape = (n_mols,n_atoms,3) 
dists = []
mol1s = []
mol2s = []
for i, mol_coords in enumerate(all_atom_coords):
    temp_dist = []
    for x in range(len(mol_coords)):
        mol1s += [central_idx]*len(mol_coords)
        mol2s += [i]*len(mol_coords)
        disp = mol_coords - central_atom_coords # shape = (n_atoms,3)
        dist2 = disp[:,0] * disp[:,0] + disp[:,1] * disp[:,1] + disp[:,2] * disp[:,2]
        dist = np.sqrt(dist2) # shape = (n_atoms)
        temp_dist.append(dist)
        mol_coords = np.roll(mol_coords,-1,axis=0)
    dists.append(temp_dist)
dists = np.array(dists) # shape = (n_molecules,x_atoms,y_atoms) | where y in y_atoms = dist(atom_x_central, atom_y_mol_n)
# Put distances in order of atom indices
in_atom_order = np.array([dist.flatten('F') for dist in dists]).reshape(-1)
d1 = dists.shape[0]
d2 = dists.shape[1]
arange = np.arange(d2)
atom1s = np.concatenate([[x]*d2 for x in range(d2)]*d1)
atom2s = np.concatenate([np.roll(arange,-x) for x in range(d2)]*d1)
# Turn Atom Distances to DataFrame
data_dict= {'mol1s':mol1s,'mol2s':mol2s,'atom1s':atom1s,'atom2s':atom2s,'dists':in_atom_order}
atom_dist_df = pd.DataFrame(data_dict)
atom_dist_df = atom_dist_df[atom_dist_df.mol1s != atom_dist_df.mol2s] # remove central molecule

In [7]:
# Calculate Molecular Distances, Angles, Displacement
all_mols = []
for mol in acenes:
    all_mols.append(mol.centre_of_geometry())
all_mols = np.array(all_mols)
cog_disps = []
cog_dists = []
cog_mol1s = []
cog_mol2s = []
interplanar_angles = []
unit_cell_disps = []
planes = []
# Rather than making new planes every loop, make the set of planes once
for mol in acenes:
    planes.append(Plane(mol.get_backbone().atoms))
# Loop through all pairs of molecules
for i, arr1 in enumerate(all_mols[:-1]):
    for j, arr2 in enumerate(all_mols[i+1:],i+1):
        interplanar_angles.append((planes[i].plane_angle(planes[j])))
        cog_mol1s.append(i)
        cog_mol2s.append(j)
        disp = arr2 - arr1
        unit_cell_disps.append(disp)
        dist = np.sqrt(disp.dot(disp))
        cog_disps.append(disp)
        cog_dists.append(dist)
# Turn lists to arrays      
unit_cell_disps = np.array(unit_cell_disps)        
cog_dists = np.array(cog_dists)
# Create Molecule Geometry to DataFrame
data_dict= {'mol1s':cog_mol1s,'mol2s':cog_mol2s,
            'a':unit_cell_disps[:,0],'b':unit_cell_disps[:,1],'c':unit_cell_disps[:,2],
            'dists':cog_dists,'interplanar_angles':interplanar_angles}
df_cogs = np.round(pd.DataFrame(data_dict).set_index(['mol1s','mol2s']),3)

/home/harry/Documents/PhD/crystal_interactions_finder_update/PYTHON/Geometry.py:39: RuntimeWarning: invalid value encountered in arccos
  A = np.degrees(np.arccos(d))


In [20]:
# Calculate Interactions for atom pairs whose distance is less than 6A
atom_contacts = []
red_df = atom_dist_df.loc[atom_dist_df.dists < 6]
for idx in red_df.index:
    mol1_idx = red_df.at[idx,'mol1s']
    mol2_idx = red_df.at[idx,'mol2s']
    atom1_idx = red_df.at[idx,'atom1s']
    atom2_idx = red_df.at[idx,'atom2s']
    atom1 = acenes[mol1_idx].atoms[atom1_idx]
    atom2 = acenes[mol2_idx].atoms[atom2_idx]
    coords1 = atom1.coordinates
    coords2 = atom2.coordinates
    interaction = Interaction(atom1,atom2,bases[mol1_idx])
    atom_contact = {'mol1s':mol1_idx,'mol2s':mol2_idx,'atom1s':atom1_idx,'atom2s':atom2_idx,
                    'a1':coords1[0],'b1':coords1[1],'c1':coords1[2],
                    'a2':coords2[0],'b2':coords2[1],'c2':coords2[2]}
    atom_contact.update(interaction.to_dict())
    atom_contacts.append(atom_contact)
ac_df = pd.DataFrame(atom_contacts).set_index(['mol1s','mol2s'])
# Add to df_cogs
hbond_bond = pd.DataFrame(ac_df.groupby(ac_df.index)['hydrogen_bond'].sum()).set_index(ac_df.index.unique())
pi_pi_bond = pd.DataFrame(ac_df.groupby(ac_df.index)['pi_pi_bond'].sum()).set_index(ac_df.index.unique())
halogen_bond = pd.DataFrame(ac_df.groupby(ac_df.index)['halogen_bond'].sum()).set_index(ac_df.index.unique())
ch_pi_bond = pd.DataFrame(ac_df.groupby(ac_df.index)['ch_pi_bond'].sum()).set_index(ac_df.index.unique())
hydrophobic_bond = pd.DataFrame(ac_df.groupby(ac_df.index)['hydrophobic_cc_bond'].sum()).set_index(ac_df.index.unique())
vdw_contact = pd.DataFrame(ac_df.groupby(ac_df.index)['vdw_contact'].sum()).set_index(ac_df.index.unique())
fin_df = pd.concat([df_cogs,vdw_contact,hbond_bond,pi_pi_bond,halogen_bond,ch_pi_bond,
                    hydrophobic_bond],axis=1)
ac_df.to_csv(f'test_atom_interactions.csv',index=True)
fin_df.to_csv(f'test_molecule_interactions.csv',index=True)

# Visualisation

In [26]:
atom_interactions = pd.read_csv('test_atom_interactions.csv')
molecule_interactions = pd.read_csv('test_molecule_interactions.csv')

In [49]:
def create_crystal_graph(molecule_interactions,consider_interactions='all',include_translation='True'):
    # Includes translational motifs even if the molecules do not form intermolecular interactions
    summary_df = pd.DataFrame()
    idx1 = molecule_interactions.mol1s[0]
    mol_interactions = molecule_interactions.set_index(['mol1s','mol2s'])
    mol_interactions.fillna(0,inplace=True)
    g = nx.Graph()
    g.add_node(idx1,pos=np.array([0,0,0]))
    for idx in mol_interactions.index:
        disp = mol_interactions.loc[idx,['a','b','c']].values
        if consider_interactions == 'all':
            interactions = mol_interactions.loc[idx,['vdw_contact','hydrogen_bond','pi_pi_bond',
                                                         'halogen_bond','ch_pi_bond','hydrophobic_cc_bond']]
        else:
            interactions = mol_interactions.loc[idx,consider_interactions]
        angle = mol_interactions.at[idx,'interplanar_angles']
        if idx[1] not in g:
            g.add_node(idx[1],pos=disp)
        if include_translation:
            if ((np.sum(interactions.values) > 0) | (np.sum(np.isin(np.abs(disp),0)) == 2)):
                info = interactions.to_dict()
                info.update({'angle':np.round(angle,-1)})
                g.add_edge(idx[0],idx[1],info=info)      
        else:
            if np.sum(interactions.values) > 0:
                info = interactions.to_dict()
                info.update({'angle':np.round(angle,-1)})
                g.add_edge(idx[0],idx[1],info=info)
            
    return g

In [43]:
def network_plot_3D(G, angle, save=False):
    colours = ['black',
               'firebrick',
               'sandybrown',
               'orange',
               'gold',
               'lawngreen',
               'forestgreen',
               'mediumturquoise'
               'dodgerblue',
               'lightslategray',
               'navy',
               'blueviolet',
               'fuchsia',
               'pink']
    # Get node positions
    pos = nx.get_node_attributes(G, 'pos')
    # Get number of nodes
    n = G.number_of_nodes()
    # Get the maximum number of edges adjacent to a single node
    edge_max = max([G.degree(key) for key in pos.keys()])
    # Define color range proportional to number of edges adjacent to a single node
    colors = [plt.cm.plasma(G.degree(key)/edge_max) for key in pos.keys()] 
    # Get edge type indices for each edge
    unique_edges = []
    edge_info = []
    for x in G.edges.data('info'):
        unique_edges.append(x[2]) if x[2] not in edge_info else 0 
        edge_info.append(x[2])
    edge_types = [unique_edges.index(edge[2]) for edge in G.edges.data('info')]
    # 3D network plot
    with plt.style.context(('ggplot')):
        
        fig = plt.figure(figsize=(12,8),dpi=100)
        ax = Axes3D(fig)
        
        # Loop on the pos dictionary to extract the x,y,z coordinates of each node
        counter = 0
        
        for key, value in pos.items():
            xi = value[0]
            yi = value[1]
            zi = value[2]
            
            # Scatter plot
            ax.scatter(xi, yi, zi, color='black', s=20, edgecolors='k', alpha=0.7)
            counter += 1
        
        # Loop on the list of edges to get the x,y,z, coordinates of the connected nodes
        # Those two points are the extrema of the line to be plotted
        edges_encountered = []
        for i,j in enumerate(G.edges()):
            x = np.array((pos[j[0]][0], pos[j[1]][0]))
            y = np.array((pos[j[0]][1], pos[j[1]][1]))
            z = np.array((pos[j[0]][2], pos[j[1]][2]))
        
        # Plot the connecting lines
            if edge_types[i] not in edges_encountered:
                ax.plot(x, y, z, c=colours[edge_types[i]],alpha=0.5, label=edge_info[i])
                edges_encountered.append(edge_types[i])
            else:
                ax.plot(x, y, z, c=colours[edge_types[i]], alpha=0.5)
        plt.legend()
    
    # Set the initial view
    ax.view_init(30, angle)
    # Hide the axes
    
    plt.show()
    
    return

In [51]:
G = create_crystal_graph(molecule_interactions,consider_interactions=['pi_pi_bond'],include_translation=False)

In [52]:
max([G.degree(node) for node in G.nodes])

1

In [53]:
edge_info = []
for x in G.edges.data('info'):
    edge_info.append(x[2]) if x[2] not in edge_info else 0 
print(edge_info)
print(len(edge_info))

[{'pi_pi_bond': 10.0, 'angle': 0.0}]
1


In [56]:
network_plot_3D(G, 0, save=False)